In [ ]:
import tiny_model
from tiny_model import TinyModel
from datasets import load_dataset
import tokre
from utils import *

tokre.setup(tokenizer=tiny_model.tokenizer)
import json

MLP_DIR = 'mlp_map_test'
MLP_NAME = 'M0_S-2_R1_P0'
TOT_FEATS = 9662
CORES_PER_ACTOR = 6

import numpy as np
from utils import all_tok_ids
import os
from tqdm import tqdm
import ray
import time


MLP_DIR = 'mlp_map_test'
MLP_NAME = 'M0_S-2_R1_P0'
feat_idx = 1000

acts = load_sparse_feat_acts(f'mlp_map_test/{MLP_NAME}/{feat_idx}.pt')

2024-07-06 15:09:27,168	INFO worker.py:1744 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 
/Users/noanabeshima/fast_feat_labelling/utils.py:55: UserWarning: Sparse CSR tensor support is in beta state. If you miss a functionality in the sparse tensor support, please submit a feature request to https://github.com/pytorch/pytorch/issues. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/SparseCsrTensorImpl.cpp:55.)
  feat_acts = torch.sparse_csr_tensor(**csr_kwargs, size=(csr_kwargs['crow_indices'].shape[0]-1, 128))


(raylet) [2024-07-06 15:14:02,383 E 8109 7775394] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-07-06_15-09-25_480314_8094 is over 95% full, available space: 49499709440; capacity: 994662584320. Object creation will fail if spilling is required.


In [10]:
import numpy as np


tensor(0.0008)

: 

In [4]:
acts.shape

torch.Size([5885, 128])

In [70]:
def slice_csr(csr, start=0, end=None):
    '''
    slice csr along the first dimension
    '''
    if start is None:
        start = 0
    
    new_crow_indices = csr.crow_indices()[start:None if end is None else end] - csr.crow_indices()[start]
    val_start = csr.crow_indices()[start].item()
    val_end = None if end is None else csr.crow_indices()[end].item()

    new_col_indices=csr.col_indices()[val_start:val_end]
    new_values = csr.values()[val_start:val_end]

    if end is None:
        new_size = (csr.size()[0]-start, csr.size()[1])
    else:
        new_size = (end - start, csr.size()[1])
    
    return torch.sparse_csr_tensor(
        crow_indices=new_crow_indices,
        col_indices=new_col_indices,
        values=new_values,
        size=new_size
    )


    
sliced = slice_csr(acts, 10, None)


: 

In [7]:
acts[0][4]

tensor(0.)

In [13]:
acts[9,58]

tensor(0.)

In [14]:
errs.shape

torch.Size([10, 128])

In [38]:
acts.to_dense().shape

torch.Size([5885, 128])

In [69]:
errs = (slice_csr(acts, 5, None).to_dense() - acts.to_dense()[5:])

errs.nonzero()

15 None


tensor([], size=(0, 2), dtype=torch.int64)